In [139]:
import numpy as np
import scipy.spatial as spatial
import pandas as pd
import pickle
import os
from smart_open import smart_open
import gensim

In [140]:
%matplotlib inline

In [141]:
groundTruthFile='/Users/eslt0101/Projects/EviDENce/GroundTruth/Manual_annotation_TOTAAL_door_Jeroen_en_Susan.xlsx'
groundTruthFileSheet='Resultaat Manuele Annotatie'
groundTruth = pd.read_excel(groundTruthFile, sheet_name=groundTruthFileSheet)


#groundTruth['Titel']


#type(groundTruth)

groundTruth.drop_duplicates(['Titel'],keep='first', inplace=True)
groundTruth.reset_index(drop=True,inplace=True)
#groundTruth['Titel'][121]


In [142]:
stopwordsFile = '/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded_gensim_processed.txt'
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output_clean/TargetSize150'
corpus_token_file = 'GV_corpus_text_para_clean.txt'
corpus_text_file = 'GV_corpus_text_para_clean.txt'
corpus_filenames_file = 'GV_corpus_text_para_clean_filenames.txt'

full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames_file) 

with open(stopwordsFile) as f:
    dutch_stopwords = f.readlines()
    

dutch_stopwords = [w.rstrip() for  w in dutch_stopwords]


def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1


corp = list(read_corpus(full_ctokf,labeled=True))


corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec_clean.npy')




In [143]:
with open(full_cff,'r') as fnf:
    corpFileNames=fnf.readlines()

corpFileNames = [fn.rstrip().split('.')[0] for fn in corpFileNames ]
#corpFileNames = [fn.rstrip() for fn in corpFileNames ]


groundTruth_inCorpusIds =[]
for j in range(len(groundTruth)):
    #print(j, groundTruth['Titel'][j])
    gtTitel = groundTruth['Titel'][j]
    corpusIndex = [i for i,ent in enumerate(corpFileNames) if ent == gtTitel]
    groundTruth_inCorpusIds.append(corpusIndex[0])



#groundTruth_inCorpusIds




gt_corp_lookup = [corp_lookup[i] for i in groundTruth_inCorpusIds]



gt_corp=[corp[i] for i in groundTruth_inCorpusIds]


In [144]:
gt_corp_reinf_medvec=reinf_corp_medvec[groundTruth_inCorpusIds]


### Append reinferred vectors to dataframe

series_reinfVec =pd.Series(np.array([]))
for i in range(len(gt_corp_reinf_medvec)):
    series_reinfVec[i] = gt_corp_reinf_medvec[i]
    


groundTruthEx = groundTruth



groundTruthEx.insert(10,'reinfVec',series_reinfVec)




In [145]:
#groundTruthEx = pd.read_csv('/Users/eslt0101/Projects/REPOS/eScience/EviDENce/doc2vec/d2v_evaluation/groundTruthEx_pd.csv')#

In [146]:
groundTruthEx


,Herkomst,Titel,Tekst,JEROEN_janee,JEROEN_OPM,SUSAN_janee,SUSAN_OPM,VERGELIJKING,CONCLUSIE,EINDOORDEEL_janee,reinfVec
0,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,"Rotterdammer bent , maar dat ben ik dus niet ,...",nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.3392736315727234, 0.27179042994976044, -0.1..."
1,manual_annotation_sample_seed1111,GV_NMKV_Sachsenhausen_01_conversation_clipped_...,Op de blokken stond geschilderd : es gibt nur ...,ja,indirect,ja,twijfel,zelfde,NaN,ja,"[-0.16284064203500748, 0.10027723386883736, -0..."
2,manual_annotation_sample_seed1111,GV_NIOD_Buchenwald_36_clipped_150_paragraph_55...,En die een kader te ja . Dat was een aantal ap...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.042901355773210526, 0.13078030943870544, 0...."
3,manual_annotation_sample_seed1111,GV_Overloon_evacuees_08_conversation_clipped_1...,Die onderduiker die met die koe aankwam dat wa...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.2558271139860153, -0.16466810554265976, -0...."
4,manual_annotation_sample_seed1111,GV_Limburg_gastkind_03_conversation_clipped_15...,Hebben jullie er na de oorlog nog veel over ge...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[-0.032807089388370514, 0.14076124876737595, -..."
5,manual_annotation_sample_seed1111,GV_Zigma_koopvaardij_07_conversation_clipped_1...,In ieder konvooi had je een paar van die schee...,ja,NaN,ja,NaN,zelfde,NaN,ja,"[0.4917626976966858, 0.24750546365976334, -0.0..."
6,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Smits_con...,"En als je een schoen aan hebt , dat is een hee...",ja,NaN,ja,NaN,zelfde,NaN,ja,"[-0.006696479162201285, 0.1732935756444931, -0..."
7,manual_annotation_sample_seed1111,GV_SiteFilms_Java_05_conversation_clipped_150_...,Maar hoe ze heetten ... Dat is van korte duur ...,nee,NaN,nee,twijfel,zelfde,NaN,nee,"[0.045544352382421494, 0.2830629199743271, 0.1..."
8,manual_annotation_sample_seed1111,GV_KNMG_medici_11_conversation_clipped_150_par...,O u besprak de patiënten met Weterings ? Jazek...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.32386012375354767, 0.21032172441482544, -0...."
9,manual_annotation_sample_seed1111,GV_OVCG_Groningeninoorlogstijd_08_conversation...,Dus . Toen kwam er in een keer een hele andere...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.06386139988899231, 0.307253435254097, -0.11..."


In [167]:
def retrieval_pass(inVec, corpDF, nReturn):
    """calculate cosine ditance of query vector inVec to all corpus vectors"""
    cosd_vec_corp = cosdistance_corp(inVec,corpDF['reinfVec'])
    
    """rank cosine distances in descending order"""
    ranked_retrieval = np.argsort(cosd_vec_corp)[::-1]
    
    "get cosine distances in ranked order"
    ranked_cosd = cosd_vec_corp[ranked_retrieval]
    ret_cosd = cosd_vec_corp[ranked_retrieval]
    
    
    
    """ return top nReturn instances"""
    ranking = corpDF.iloc[ranked_retrieval,[1,9,10]].copy()
    ranking['cosine_distance']= ret_cosd
    

    
    selected = ranking.iloc[0:nReturn].copy()
    """ create pandas series of selected ranked cosine distances"""
    #ret_cosd = pd.Series(ranked_cosd[0:nReturn])
    
    """insert cosine distance into returned ranking"""
    
    
    return selected
    
    
    

In [168]:
def cosdistance(v1,v2):
    d = np.array([1. - spatial.distance.cosine(v1,v2)])
    return d

In [169]:
def cosdistance_corp(vec,corpus_vectors):
    cosd_vec_corp =[]
    for i in range (len(corpus_vectors)):
        cd = cosdistance(vec,corpus_vectors[i])
        cosd_vec_corp.append(cd)
    cosd_vec_corp = np.array(cosd_vec_corp).flatten()
    return cosd_vec_corp

In [189]:
def run_evaluation(corpDF):
    
    for i in range(1):
        queryVec = corpDF['reinfVec'][i]
        queryTitle = corpDF['Titel'][i]
        
        incorp = corpDF.copy()
        incorp.drop([i],inplace=True)
        
        
        
    return incorp

In [190]:
test =run_evaluation(groundTruthEx)

In [188]:
test

,Herkomst,Titel,Tekst,JEROEN_janee,JEROEN_OPM,SUSAN_janee,SUSAN_OPM,VERGELIJKING,CONCLUSIE,EINDOORDEEL_janee,reinfVec
0,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,"Rotterdammer bent , maar dat ben ik dus niet ,...",nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.3392736315727234, 0.27179042994976044, -0.1..."
2,manual_annotation_sample_seed1111,GV_NIOD_Buchenwald_36_clipped_150_paragraph_55...,En die een kader te ja . Dat was een aantal ap...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.042901355773210526, 0.13078030943870544, 0...."
3,manual_annotation_sample_seed1111,GV_Overloon_evacuees_08_conversation_clipped_1...,Die onderduiker die met die koe aankwam dat wa...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.2558271139860153, -0.16466810554265976, -0...."
4,manual_annotation_sample_seed1111,GV_Limburg_gastkind_03_conversation_clipped_15...,Hebben jullie er na de oorlog nog veel over ge...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[-0.032807089388370514, 0.14076124876737595, -..."
5,manual_annotation_sample_seed1111,GV_Zigma_koopvaardij_07_conversation_clipped_1...,In ieder konvooi had je een paar van die schee...,ja,NaN,ja,NaN,zelfde,NaN,ja,"[0.4917626976966858, 0.24750546365976334, -0.0..."
6,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Smits_con...,"En als je een schoen aan hebt , dat is een hee...",ja,NaN,ja,NaN,zelfde,NaN,ja,"[-0.006696479162201285, 0.1732935756444931, -0..."
7,manual_annotation_sample_seed1111,GV_SiteFilms_Java_05_conversation_clipped_150_...,Maar hoe ze heetten ... Dat is van korte duur ...,nee,NaN,nee,twijfel,zelfde,NaN,nee,"[0.045544352382421494, 0.2830629199743271, 0.1..."
8,manual_annotation_sample_seed1111,GV_KNMG_medici_11_conversation_clipped_150_par...,O u besprak de patiënten met Weterings ? Jazek...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.32386012375354767, 0.21032172441482544, -0...."
9,manual_annotation_sample_seed1111,GV_OVCG_Groningeninoorlogstijd_08_conversation...,Dus . Toen kwam er in een keer een hele andere...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.06386139988899231, 0.307253435254097, -0.11..."
10,manual_annotation_sample_seed1111,GV_Verhalis_kloosterzusters_07b_conversation_c...,Goed we zijn weer klaar zuster Martha . Ja . E...,nee,"Alleen het woord ""oorlog"" wordt gebruikt; geen...",nee,NaN,zelfde,NaN,nee,"[0.2551063448190689, 0.11829375475645065, -0.1..."


In [179]:
groundTruthEx

,Herkomst,Titel,Tekst,JEROEN_janee,JEROEN_OPM,SUSAN_janee,SUSAN_OPM,VERGELIJKING,CONCLUSIE,EINDOORDEEL_janee,reinfVec
0,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,"Rotterdammer bent , maar dat ben ik dus niet ,...",nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.3392736315727234, 0.27179042994976044, -0.1..."
1,manual_annotation_sample_seed1111,GV_NMKV_Sachsenhausen_01_conversation_clipped_...,Op de blokken stond geschilderd : es gibt nur ...,ja,indirect,ja,twijfel,zelfde,NaN,ja,"[-0.16284064203500748, 0.10027723386883736, -0..."
2,manual_annotation_sample_seed1111,GV_NIOD_Buchenwald_36_clipped_150_paragraph_55...,En die een kader te ja . Dat was een aantal ap...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.042901355773210526, 0.13078030943870544, 0...."
3,manual_annotation_sample_seed1111,GV_Overloon_evacuees_08_conversation_clipped_1...,Die onderduiker die met die koe aankwam dat wa...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.2558271139860153, -0.16466810554265976, -0...."
4,manual_annotation_sample_seed1111,GV_Limburg_gastkind_03_conversation_clipped_15...,Hebben jullie er na de oorlog nog veel over ge...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[-0.032807089388370514, 0.14076124876737595, -..."
5,manual_annotation_sample_seed1111,GV_Zigma_koopvaardij_07_conversation_clipped_1...,In ieder konvooi had je een paar van die schee...,ja,NaN,ja,NaN,zelfde,NaN,ja,"[0.4917626976966858, 0.24750546365976334, -0.0..."
6,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Smits_con...,"En als je een schoen aan hebt , dat is een hee...",ja,NaN,ja,NaN,zelfde,NaN,ja,"[-0.006696479162201285, 0.1732935756444931, -0..."
7,manual_annotation_sample_seed1111,GV_SiteFilms_Java_05_conversation_clipped_150_...,Maar hoe ze heetten ... Dat is van korte duur ...,nee,NaN,nee,twijfel,zelfde,NaN,nee,"[0.045544352382421494, 0.2830629199743271, 0.1..."
8,manual_annotation_sample_seed1111,GV_KNMG_medici_11_conversation_clipped_150_par...,O u besprak de patiënten met Weterings ? Jazek...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.32386012375354767, 0.21032172441482544, -0...."
9,manual_annotation_sample_seed1111,GV_OVCG_Groningeninoorlogstijd_08_conversation...,Dus . Toen kwam er in een keer een hele andere...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.06386139988899231, 0.307253435254097, -0.11..."


In [171]:
retlist_2 = retrieval_pass(groundTruthEx['reinfVec'][227],groundTruthEx,183)

,Titel,EINDOORDEEL_janee,reinfVec,cosine_distance
227,GV_DeJager_ReisvandeRazzia_interview_van_Esch_...,ja,"[0.4200042337179184, 0.2546316534280777, -0.06...",1.000000
32,01_JKKV_2003_LOTTY_HUFFENER-VEFFER_conversatio...,ja,"[0.48686669766902924, 0.27796636521816254, -0....",0.922404
896,GV_MUMA_Molukkers_48_clipped_150_paragraph_98-...,nee,"[0.4899682551622391, 0.23521731793880463, -0.1...",0.919731
687,GV_Verhalis_kloosterzusters_07c_conversation_c...,nee,"[0.5865635275840759, 0.17963603138923645, -0.0...",0.910667
124,GV_Nijkerk_Vluchtelingen_02_conversation_clipp...,nee,"[0.18168242275714874, 0.06366529501974583, -0....",0.909889
38,GV_Zigma_koopvaardij_06_conversation_clipped_1...,nee,"[0.4335503578186035, 0.2419842928647995, -0.23...",0.909609
379,GV_Groningen_Scholtenhuis_01_conversation_clip...,nee,"[0.371928334236145, 0.4519408792257309, -0.243...",0.900303
574,GV_DeJager_ReisvandeRazzia_interview_Woudstra_...,nee,"[0.48545728623867035, 0.28663522005081177, -0....",0.896757
884,GV_GAR_bombardement_03_conversation_clipped_15...,nee,"[0.1643867790699005, 0.3686671853065491, -0.26...",0.896417
465,GV_MUMA_Molukkers_31b_clipped_150_paragraph_40...,nee,"[0.2724558860063553, 0.20027587562799454, -0.2...",0.895353


In [97]:
aa=np.array(groundTruthEx['reinfVec'][0])
aa


array([ 0.33927363,  0.27179043, -0.16801026, -0.20380647,  0.13683586,
        0.08472264,  0.18424188, -0.18145876,  0.22445451, -0.00311114,
        0.01058206,  0.12031288, -0.26398021,  0.33625874,  0.03873563,
       -0.05989163,  0.05653091,  0.16971132,  0.15461932, -0.02199169,
       -0.36243436,  0.04165543,  0.04841177, -0.02837305,  0.09280476,
        0.23569371,  0.17932937,  0.08360341, -0.1932203 ,  0.04128065,
       -0.13684578, -0.11732881,  0.45511979, -0.1771264 ,  0.11421824,
        0.0995436 ,  0.05680378,  0.13380276,  0.07739257,  0.10761983,
        0.07975015,  0.04996086, -0.05538582, -0.02385679, -0.19928084,
       -0.37455796,  0.15882851,  0.51568201, -0.1577179 , -0.18011839])

In [98]:
bb = groundTruthEx['reinfVec'][1]
bb

array([-0.16284064,  0.10027723, -0.38268279, -0.15086383, -0.01915314,
        0.26706837, -0.13030646, -0.19925655,  0.31914362,  0.03294852,
       -0.04579653, -0.09568018, -0.25194153, -0.15989885, -0.13336261,
       -0.04517514,  0.2105265 ,  0.08377888,  0.10617176, -0.10606162,
       -0.27820943, -0.03878747, -0.09202768,  0.27390203,  0.19896287,
        0.38577724,  0.28559934,  0.13979103, -0.14089702,  0.08763998,
       -0.15093483, -0.31407635,  0.30477685,  0.24806837, -0.09777014,
        0.04408616,  0.27188371, -0.10986643,  0.2180142 ,  0.31456161,
        0.21628446,  0.20836323,  0.208125  , -0.143571  , -0.15108767,
        0.07543454, -0.1319905 ,  0.35731459,  0.05840232, -0.40044154])

In [100]:
cc=cosdistance_corp(aa,groundTruthEx['reinfVec'])


In [103]:
id = np.argsort(cc)[::-1]

In [104]:
cc[id]

array([ 1.        ,  0.8575179 ,  0.8534164 ,  0.84780629,  0.84720767,
        0.84437662,  0.84111166,  0.84106692,  0.84096804,  0.838482  ,
        0.8348534 ,  0.83284815,  0.83269166,  0.83116774,  0.82172381,
        0.82127158,  0.81815292,  0.81629879,  0.81440174,  0.81365487,
        0.81271439,  0.81201956,  0.81182769,  0.81148147,  0.80742736,
        0.80558857,  0.79761985,  0.79483138,  0.79332068,  0.78813935,
        0.78812488,  0.78556638,  0.78534383,  0.78291146,  0.78073793,
        0.77958482,  0.7794398 ,  0.77918836,  0.77894893,  0.77889533,
        0.77852459,  0.77846187,  0.77821274,  0.77788193,  0.7771388 ,
        0.7742028 ,  0.77283232,  0.77273796,  0.77218187,  0.77217427,
        0.77138819,  0.77062376,  0.77007763,  0.76849282,  0.7684555 ,
        0.76833723,  0.76825412,  0.76716343,  0.76675148,  0.76657627,
        0.76616051,  0.7657976 ,  0.76503451,  0.76371216,  0.76360207,
        0.76354948,  0.76332657,  0.76286907,  0.76232635,  0.76

In [106]:
dd = groundTruthEx.iloc[id[0:10],[1,9,10]].copy()

In [107]:
dd


,Titel,EINDOORDEEL_janee,reinfVec
0,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,nee,"[0.3392736315727234, 0.27179042994976044, -0.1..."
206,GV_DeJager_ReisvandeRazzia_interview_Goudappel...,nee,"[0.45767350494861603, 0.28337402641773224, -0...."
227,GV_DeJager_ReisvandeRazzia_interview_van_Esch_...,ja,"[0.4200042337179184, 0.2546316534280777, -0.06..."
522,GV_Zigma_zeemansgezin_02_conversation_clipped_...,nee,"[0.51364865899086, 0.21421899646520615, -0.058..."
38,GV_Zigma_koopvaardij_06_conversation_clipped_1...,nee,"[0.4335503578186035, 0.2419842928647995, -0.23..."
574,GV_DeJager_ReisvandeRazzia_interview_Woudstra_...,nee,"[0.48545728623867035, 0.28663522005081177, -0...."
268,GV_Kindermonument_Gaaspstraat_02_conversation_...,nee,"[0.40550200641155243, 0.12081876024603844, -0...."
880,GV_Schiedam_gastgezin2_06_conversation_clipped...,nee,"[0.467038556933403, 0.47100353240966797, -0.27..."
836,GV_SiteFilms_Java_03_conversation_clipped_150_...,nee,"[0.44200193881988525, 0.12589943408966064, -0...."
123,01_JKKV_2003_LOTTY_HUFFENER-VEFFER_conversatio...,ja,"[0.40477579832077026, 0.16137678176164627, -0...."
